In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue May 21 04:32:53 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
# !cd /content/drive/My\ Drive/project2 && wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
# > /dev/null 2>&1
rm -r food-101

In [ ]:
!tar -xvf /content/drive/My\ Drive/project2/food-101.tar.gz > /dev/null 2>&1

In [ ]:
%%capture
import os
from collections import defaultdict
import shutil, sys 

if not os.path.isdir('/content/food-101/test') or not os.path.isdir('/content/food-101/train'):

    def copytree(src, dst, symlinks = False, ignore = None):
        if not os.path.exists(dst):
            os.makedirs(dst)
            shutil.copystat(src, dst)
        lst = os.listdir(src)
        if ignore:
            excl = ignore(src, lst)
            lst = [x for x in lst if x not in excl]
        for item in lst:
            s = os.path.join(src, item)
            d = os.path.join(dst, item)
            if symlinks and os.path.islink(s):
                if os.path.lexists(d):
                    os.remove(d)
                os.symlink(os.readlink(s), d)
                try:
                    st = os.lstat(s)
                    mode = stat.S_IMODE(st.st_mode)
                    os.lchmod(d, mode)
                except:
                    pass # lchmod not available
            elif os.path.isdir(s):
                copytree(s, d, symlinks, ignore)
            else:
                shutil.copy2(s, d)

    def generate_dir_file_map(path):
        dir_files = defaultdict(list)
        with open(path, 'r') as txt:
            files = [l.strip() for l in txt.readlines()]
            for f in files:
                dir_name, id = f.split('/')
                dir_files[dir_name].append(id + '.jpg')
        return dir_files

    train_dir_files = generate_dir_file_map('/content/food-101/meta/train.txt')
    test_dir_files = generate_dir_file_map('/content/food-101/meta/test.txt')


    def ignore_train(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = train_dir_files[subdir]
        return to_ignore

    def ignore_test(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = test_dir_files[subdir]
        return to_ignore

    copytree('/content/food-101/images', '/content/food-101/test', ignore=ignore_train)
    copytree('/content/food-101/images', '/content/food-101/train', ignore=ignore_test)
    
else:
    print('Train/Test files already copied into separate folders.')

In [ ]:
# !rm -r /content/drive/My\ Drive/project2/food-101/test

In [ ]:
os.path.isdir('/content/food-101/train')

True

In [ ]:
os.path.isdir('/content/food-101/test')

True

In [ ]:
from __future__ import print_function
from __future__ import division

from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model
from keras.layers import Dense
from keras.layers import Input
import numpy as np
import argparse


Using TensorFlow backend.


In [ ]:
def setup_generator(train_path, test_path, batch_size, dimentions):
    train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_path,  # this is the target directory
        target_size=dimentions,
        batch_size=batch_size)

    validation_generator = test_datagen.flow_from_directory(
        test_path, # this is the target directory
        target_size=dimentions,
        batch_size=batch_size)

    return train_generator, validation_generator

In [ ]:
def load_image(img_path, dimentions, rescale=1. / 255):
    img = image.load_img(img_path, target_size=dimentions)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x *= rescale # rescale the same as when trained

    return x

In [ ]:
# def create_model(num_classes, dropout, shape):
#     base_model = ResNet50(
#         weights='imagenet',
#         include_top=False,
#         input_tensor=Input(
#             shape=shape))
# #     for l in base_model.layers[:-10]:
# #         l.trainable = False
#     x = base_model.output
#     x = GlobalAveragePooling2D()(x)
#     x = Dropout(dropout)(x)
#     predictions = Dense(num_classes, activation='softmax')(x)

#     model_final = Model(inputs=base_model.input, outputs=predictions)
#     model_final.summary()
# #     model_final.load_weights("/content/drive/My Drive/project2/weights-resnet50.epoch-06-val_loss-1.14.hdf5")
#     return model_final

In [ ]:
# def train_model(model_final, train_generator, validation_generator, callbacks):
#     model_final.compile(
#         loss='categorical_crossentropy',
#         optimizer='adam',
#         metrics=['accuracy'])

#     model_final.fit_generator(train_generator, validation_data=validation_generator,
#                               epochs=10, callbacks=callbacks,
#                               steps_per_epoch=train_generator.samples//32,
#                               validation_steps=validation_generator.samples//32)

In [ ]:
shape = (224, 224, 3)
X_train, X_test = setup_generator('/content/food-101/train', '/content/food-101/test', 32, shape[:2])


Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [ ]:
X_train.num_classes

101

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.layers import Input


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.regularizers import l2
import keras.backend as K
import math

base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
x = base_model.output
x = AveragePooling2D()(x)

x = Dropout(.5)(x)
x = Flatten()(x)
predictions = Dense(X_train.num_classes, init='glorot_uniform', W_regularizer=l2(.0005), activation='softmax')(x)

model = Model(input=base_model.input, output=predictions)

opt = SGD(lr=.1, momentum=.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/project2/lan2_v/model4_lan2.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('/content/drive/My Drive/project2/lan2_v/model4.log')

def schedule(epoch):
    if epoch < 5:
        return 0.001
    elif epoch < 10:
        return .0002
    elif epoch < 15:
        return 0.00002
    else:
        return .0000005
lr_scheduler = LearningRateScheduler(schedule)
model.summary()



Instructions for updating:
Colocations handled automatically by placer.
87916544/87910968 [==============================] - 1s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
_______________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(101, activation="softmax", kernel_initializer="glorot_uniform", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [ ]:
model = load_model(filepath='/content/drive/My Drive/project2/model4.18-0.81.hdf5')

model.fit_generator(X_train, validation_data=X_test,
                              epochs=50,
                              steps_per_epoch=X_train.samples//32,
                              validation_steps=X_test.samples//32,
                               callbacks=[lr_scheduler, csv_logger, checkpointer])

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
2367/2367 [==============================] - 1663s 702ms/step - loss: 0.4592 - acc: 0.8927 - val_loss: 0.8255 - val_acc: 0.8284

Epoch 00001: val_loss improved from inf to 0.82546, saving model to /content/drive/My Drive/project2/lan2_v/model4_lan2.01-0.83.hdf5
Epoch 2/50
2367/2367 [==============================] - 1646s 695ms/step - loss: 0.4347 - acc: 0.9007 - val_loss: 0.8175 - val_acc: 0.8283

Epoch 00002: val_loss improved from 0.82546 to 0.81745, saving model to /content/drive/My Drive/project2/lan2_v/model4_lan2.02-0.82.hdf5
Epoch 3/50
2367/2367 [==============================] - 1633s 690ms/step - loss: 0.4157 - acc: 0.9057 - val_loss: 0.8181 - val_acc: 0.8273

Epoch 00003: val_loss did not improve from 0.81745
Epoch 4/50
2367/2367 [==============================] - 1591s 672ms/step - loss: 0.4048 - acc: 0.9063 - val_loss: 0.8290 - val_acc: 0.8279

Epoch 00004: val_loss did not improve from 0.81745
Epoch 5/50
2367/2367